In [1]:
from pyspark.ml import pipeline
from pyspark.ml.classification import RandomForestClassifier,GBTClassifier,DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler

In [2]:
un=sqlContext.sql("SELECT * FROM college")

In [3]:
un.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: double (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)

In [4]:
un.columns

Out[ 5 ]: 
['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [5]:
un.head(1)

Out[ 7 ]: [Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [6]:
assembler=VectorAssembler(inputCols=['Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'],outputCol='features')

In [7]:
out=assembler.transform(un)
from pyspark.ml.feature import StringIndexer

In [8]:
indexer=StringIndexer(inputCol="Private",outputCol="index")
output=indexer.fit(out).transform(out)

In [9]:
ou=output.select("features","index")

In [10]:
train,test=ou.randomSplit([0.65,0.35])

In [11]:
from pyspark.ml import Pipeline
dtc=DecisionTreeClassifier(labelCol="index",featuresCol="features")
rfc=RandomForestClassifier(labelCol="index",featuresCol="features")
gbt=GBTClassifier(labelCol="index",featuresCol="features")

In [12]:
dmod=dtc.fit(train)
rmod=rfc.fit(train)
gmod=gbt.fit(train)

In [13]:
dres=dmod.transform(test)
rres=rmod.transform(test)
gres=gmod.transform(test)

In [14]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
eval=BinaryClassificationEvaluator(labelCol='index')

In [15]:
print("DTC")
print(eval.evaluate(dres))

DTC
0.9139222483595394

In [16]:
print("RFC")
print(eval.evaluate(rres))

RFC
0.9806858982295421

In [17]:
print("GDT")
print(eval.evaluate(gres))

GDT
0.9733193017209362

In [18]:
eval2=BinaryClassificationEvaluator(labelCol='index',rawPredictionCol='prediction')
print("GDT")
print(eval2.evaluate(gres))

GDT
0.9024699764764145

In [19]:
rfc2=RandomForestClassifier(labelCol="index",featuresCol="features",numTrees=200)
rmod2=rfc2.fit(train)
rres2=rmod2.transform(test)

In [20]:
print("RFC")
print(eval.evaluate(rres2))

RFC
0.9816144608146599